In [1]:
import pyspark 
from pyspark.sql  import SparkSession 
from pyspark.sql.functions import collect_list, col, lit 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType, DateType, ArrayType
import os 
import json 
from pyspark.sql.functions import to_date, when
from urllib.parse import urlparse

In [24]:
# !wget https://repo1.maven.org/maven2/org/apache/hive/hive-metastore/2.3.9/hive-metastore-2.3.9-sources.jar

# !wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.4_2.12/1.5.0/iceberg-spark-runtime-3.4_2.12-1.5.0.jar
# !wget https://repo1.maven.org/maven2/org/projectnessie/nessie-integrations/nessie-spark-extensions-3.4_2.12/0.77.0/nessie-spark-extensions-3.4_2.12-0.77.0.jar
# !wget https://repo1.maven.org/maven2/software/amazon/awssdk/bundle/2.24.8/bundle-2.24.8.jar
# !wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-nessie/1.4.0/iceberg-nessie-1.4.0.jar
# !wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar
# !wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/3.3.4/hadoop-common-3.3.4.jar
# !wget https://repo1.maven.org/maven2/io/minio/minio/8.5.0/minio-8.5.0.jar
# !wget https://repo1.maven.org/maven2/software/amazon/awssdk/url-connection-client/2.24.8/url-connection-client-2.24.8.jar
# !wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.207/aws-java-sdk-bundle-1.12.207.jar
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-hadoop-cloud_2.13/3.2.1/spark-hadoop-cloud_2.13-3.2.1.jar

--2024-12-11 10:53:16--  https://repo1.maven.org/maven2/org/apache/spark/spark-hadoop-cloud_2.13/3.2.1/spark-hadoop-cloud_2.13-3.2.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 146.75.92.209, 2a04:4e42:87::209
Connecting to repo1.maven.org (repo1.maven.org)|146.75.92.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 602385 (588K) [application/java-archive]
Saving to: ‘spark-hadoop-cloud_2.13-3.2.1.jar’

spark-hadoop-cloud_ 100%[===================>] 588.27K  1.21MB/s    in 0.5s    

2024-12-11 10:53:17 (1.21 MB/s) - ‘spark-hadoop-cloud_2.13-3.2.1.jar’ saved [602385/602385]



In [2]:
aws_access_key = "BKky9mVTl8KoPVCmZZ78"
aws_secret_key = "ZszY0hLzFcJscU8fwAg1bLYSMuWHwpyDpWwfhtmA"
aws_s3_endpoint = "http://18.140.117.93:9000" 
aws_s3_region = "us-east-1" 
nessi_warehouse = "s3a://standardize"
nessi_uri = "http://18.140.117.93:19120/api/v1"
raw_csv_path = "s3a://raw/CRMUSER_ACCOUNTS.csv"

In [3]:
conf = (
    pyspark.SparkConf()
        .setAppName('raw_to_standard_etl')
        .set('spark.jars.packages', 
             'org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.5.0,'
                'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.4_2.12:0.77.0,' 
                'software.amazon.awssdk:bundle:2.24.8,' 
                'org.apache.iceberg:iceberg-nessie:1.4.0,' 
                'org.apache.hadoop:hadoop-aws:3.3.4,'  
                'org.apache.hadoop:hadoop-common:3.3.4,'  
                'io.minio:minio:8.5.0,' 
                'org.apache.spark:spark-hadoop-cloud_2.13:3.2.1,' 
                'software.amazon.awssdk:url-connection-client:2.24.8,' 
                'com.amazonaws:aws-java-sdk-bundle:1.12.207')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', nessi_uri)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', aws_s3_endpoint)
        .set('spark.sql.catalog.nessie.warehouse', nessi_warehouse)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', aws_access_key)
        .set('spark.hadoop.fs.s3a.secret.key', aws_secret_key)
        .set('spark.hadoop.fs.s3a.endpoint', aws_s3_endpoint)
        .set('spark.hadoop.fs.s3a.region', aws_s3_region)
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 
             'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider,'
             'org.apache.hadoop.fs.s3a.EnvironmentVariableCredentialsProvider')
)

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [5]:
# Set Hadoop Configuration Explicitly
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", aws_s3_endpoint)
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")

In [6]:
# import os

# # Set environment variables before creating Spark Session
# os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
# os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key
# os.environ['AWS_S3_ENDPOINT'] = aws_s3_endpoint


In [6]:
# # Verify Credentials and Connection
# def test_s3_connection():
#     try:
#         # Attempt to list objects in the bucket
#         objects = spark.read.csv(f"s3a://raw/", header=True)
#         print("Connection successful!")
#         return True
#     except Exception as e:
#         print(f"Connection failed: {e}")
#         return False

# Read CSV with explicit options
raw_df = spark.read \
    .option("header", "true") \
    .csv("s3a://raw/CRMUSER_ACCOUNTS.csv")

In [7]:
raw_df.show()

+---------+-------+------------+---------+-------+--------------+------+---------+------+--------------+-----------------------+
|ACCOUNTID|BANK_ID|CORE_CUST_ID|   ORGKEY|ORGTYPE|          NAME|GENDER|CUST_TYPE|STATUS|PRIMARY_SOL_ID|RELATIONSHIPOPENINGDATE|
+---------+-------+------------+---------+-------+--------------+------+---------+------+--------------+-----------------------+
|        1|      1|   113391407|113391407|Account|TRAN T *** HAI|     M|    10001| ACTVE|          1404|             23-02-2018|
|        2|      1|   113391457|113391457|Account|TRAN T *** YEN|     F|    10001| ACTVE|          1003|              3/12/2018|
|        3|      1|   113391483|113391483|Account|TRAN T *** BAC|     F|    10001| ACTVE|          2201|             15-06-2018|
|        4|      1|   105260800|105260800|Account|NGUYEN *** UNG|     M|    10001| ACTVE|          1401|             16-11-2018|
|        5|      1|   106573010|106573010|Account|TRAN N *** ANH|     M|    10001| ACTVE|        